## Purpose

- Async pull latest kitties that are for sale with given price boundaries 
- Should I pull in lots of 2000? Then I can make predictions on! I think so 
- Should also combine the latest pull of id_s with pulling of traits
- create df that is model ready (has latest trends and ETH price)
- how to deal with get_dummies?
- Need latest ETH TO USD
- 80000000000000000 = 0.08 ETH (divide by 1000000000000000000)



In [8]:
9711936536387730/1000000000000000000

0.00971193653638773

In [1]:
import requests
import json
import pandas as pd
import codecs
pd.set_option("max_columns", 30)
import os

In [13]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [14]:
ETH = pd.read_csv("../Data/ETH_price.csv")
ETH.head()

,date,open,high,low,close,volume,marketcap
0,2019-07-06,287.89,295.99,286.67,287.55,7674615025,30707785756
1,2019-07-05,284.38,293.92,282.69,288.00,8723493015,30751916427
2,2019-07-04,303.03,303.96,282.77,284.52,8403513106,30377082394
3,2019-07-03,291.76,303.34,291.40,303.10,9926711979,32356299765
4,2019-07-02,293.54,295.66,272.60,291.60,10618413952,31124332021


In [15]:
trends = pd.read_csv("../Data/trends.csv")
trends.head()

,date,cryptokitties
0,2019-07-04,0.544530
1,2019-07-03,0.877299
2,2019-07-02,0.514278
3,2019-07-01,0.544530
4,2019-06-30,0.423523


In [5]:
url = 'https://public.api.cryptokitties.co/v1/kitties?include=sale&price=80000000000000000-1000000000000000000&orderBy=current_price&limit=20&offset=53652&orderDirection=asc'
payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False)

parsed = json.loads(response.text)
print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "kitties": [
        {
            "auction": {
                "current_price": "1000000000000000000",
                "duration": "345600000",
                "end_price": "1000000000000000000",
                "end_time": "2018-10-27 16:24:14+00",
                "id": 1032695,
                "seller": {
                    "address": "0xcc34a5df6536a06d195cf22303dd1798c3c70af9",
                    "hasDapper": false
                },
                "start_price": "500000000000000000",
                "start_time": "2018-10-23 16:24:14+00",
                "status": "open",
                "type": "sale"
            },
            "bio": "What's up! I'm #{name}. My friends describe me as stinky and seductive. When my owner isn't watching, I steal their brooches and use them for litter paper. I'm not sorry. I think you'll love me beclaws I have cattitude.",
            "color": "babypuke",
            "created_at": "2018-01-13T11:41:46Z",
            "enhanced_cattributes":

In [6]:
all_urls = []
for i in range(int(53662/100)+ 2):
    all_urls.append(f'https://public.api.cryptokitties.co/v1/kitties?include=sale&price=80000000000000000-1000000000000000000&orderBy=current_price&limit={100}&offset={i*100}&orderDirection=asc')

In [14]:
round(int(len(all_urls))/10 - 1)

53

## Async getting latest kitties for sale!


In [32]:
import aiofiles
import os
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp


payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

async def download_html(session, url):
    async with session.request('GET', url, headers = headers, data = payload, allow_redirects=False) as res:
        filename = f'html/output_{len(os.listdir("html/"))}.html'
        print(len(os.listdir("./html")), end = "\r")
        async with aiofiles.open(filename, 'wb') as f:
            while True:
                chunk = await res.content.read(1024)
                if not chunk:
                    break
                await f.write(chunk)

        return await res.release()
    
async def main(url):
    connector = aiohttp.TCPConnector(limit=60)
    async with aiohttp.ClientSession(connector=connector) as session: #aiohttp.ClientSession()
        await download_html(session, url)

# urls = ['https://public.api.cryptokitties.co/v1/kitties/' + str(i) for i in range(100)] # a big list of websites to scrape

for i in range(int(len(all_urls)/10)+ 1):
    urls = all_urls[i*10: i*10 + 10]
    print('step: ' + str(i) + '   Number of files: ' + str(len(os.listdir("html/"))))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        asyncio.gather(*(main(url) for url in urls))
    )

step: 0   Number of files: 1
step: 1   Number of files: 11
step: 2   Number of files: 21
step: 3   Number of files: 31
step: 4   Number of files: 41
step: 5   Number of files: 51
step: 6   Number of files: 61
step: 7   Number of files: 71
step: 8   Number of files: 81
step: 9   Number of files: 91
step: 10   Number of files: 101
step: 11   Number of files: 111
step: 12   Number of files: 121
step: 13   Number of files: 130
step: 14   Number of files: 140
step: 15   Number of files: 150
step: 16   Number of files: 160
step: 17   Number of files: 170
step: 18   Number of files: 180
step: 19   Number of files: 190
step: 20   Number of files: 200
step: 21   Number of files: 210
step: 22   Number of files: 220
step: 23   Number of files: 230
step: 24   Number of files: 240
step: 25   Number of files: 250
step: 26   Number of files: 260
step: 27   Number of files: 270
step: 28   Number of files: 280
step: 29   Number of files: 290
step: 30   Number of files: 300
step: 31   Number of files: 3

### ERROR HANDLING!

In [2]:
errors = []
for i in os.listdir("html/"):
    file = codecs.open("html/" + str(i), 'r',  encoding="utf8")
    try:
        parsed = json.loads(file.read())
    except:
        pass
    
    try:
        parsed['code']
        print(i)
        errors.append(i)
    except:
        pass
        

In [3]:
payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

for e in errors:
    limit = 100
    offset = (int(e[7:].split(".")[0])-1)*100

    url = f'https://public.api.cryptokitties.co/v1/kitties?include=sale&price=80000000000000000-1000000000000000000&orderBy=current_price&limit={100}&offset={offset}&orderDirection=asc'
    response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False).text

    file = open("html/" + e, 'w', encoding="utf-8")
    file.write(response)
    file.close()

    print(e)

### PUTTING INTO DF

In [4]:
def sorting(val): 
    return (int(val[7:].split(".")[0])-1)  

dirs = sorted(os.listdir("html/"), key = sorting)
dirs[:10]

['output_0.html',
 'output_1.html',
 'output_2.html',
 'output_3.html',
 'output_4.html',
 'output_5.html',
 'output_6.html',
 'output_7.html',
 'output_8.html',
 'output_9.html']

In [5]:
id_s = []
for d in dirs:
    file= codecs.open("html/"+ d, 'r',  encoding="utf8")
    try:
        parsed = json.loads(file.read())
    except:
        pass
    print(d + " DONE!", end = "\r")
    
    for i in range(100):
        try: 
            id_s.append(parsed['kitties'][i]['id'])
        except:
            print(str(d), end = "\r")

## NEED TO REMEMBER HOW I GOT INTO DF

In [6]:
data = pd.DataFrame(id_s, columns = ['ID'])

In [7]:
data.head(10)

,ID
0,1637950
1,1591142
2,1340038
3,1592225
4,1589771
5,1591598
6,1592451
7,1592957
8,1593101
9,1592453


In [8]:
 # Creating dataframe and columns
# data = pd.DataFrame(id_s, columns = ['ID'])

data['USD'] = ''
data['time'] = ''
# Random
data['generation'] = ''
data['color'] = ''
data['is_fancy'] = ''
data['is_exclusive'] = ''
data['fancy_type'] = ''
data['is_prestige'] = ''
data['prestige_type'] = ''
data['prestige_ranking'] = ''
data['fancy_ranking'] = ''
data['prestige_time_limit'] = ''

# Enhanced cattributes
data['enhanced_cattributes_eyes'] = '' 
data['enhanced_cattributes_pattern'] = '' 
data['enhanced_cattributes_body'] = '' 
data['enhanced_cattributes_mouth'] = '' 
data['enhanced_cattributes_colorprimary'] = '' 
data['enhanced_cattributes_colorsecondary'] = '' 
data['enhanced_cattributes_colortertiary'] = '' 
data['enhanced_cattributes_coloreyes'] = '' 

# Status
data['status_cooldown_index'] = ''
data['status_is_ready'] = ''
data['status_is_gestating'] = ''

# Purrs
data['purrs_count'] = ''
data['purrs_is_purred'] = ''

# watchlist
data['watchlist_count'] = ''
data['watchlist_is_watchlisted'] = ''


# resorting columns!
data = data[['time', 'ID', "USD"] + list(data.columns)[3:]]

In [35]:
data.head(10)

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,,1637950,8000000000000000,2,,,,,,,,,,,,,,,,,,,,,,,,
1,,1591142,8000000000000000,1,,,,,,,,,,,,,,,,,,,,,,,,
2,,1340038,8000000000000000,13,,,,,,,,,,,,,,,,,,,,,,,,
3,,1592225,8000000000000000,16,,,,,,,,,,,,,,,,,,,,,,,,
4,,1589771,8000000000000000,12,,,,,,,,,,,,,,,,,,,,,,,,
5,,1591598,8000000000000000,14,,,,,,,,,,,,,,,,,,,,,,,,
6,,1592451,8000000000000000,13,,,,,,,,,,,,,,,,,,,,,,,,
7,,1592957,8000000000000000,13,,,,,,,,,,,,,,,,,,,,,,,,
8,,1593101,8000000000000000,14,,,,,,,,,,,,,,,,,,,,,,,,
9,,1592453,8000000000000000,14,,,,,,,,,,,,,,,,,,,,,,,,


In [78]:
vals = {'USD':[], 'generation': [],'color': [], 'is_fancy':[], 'is_exclusive':[],
       'fancy_type':[], 'is_prestige':[], 'prestige_type':[], 'prestige_ranking':[],
       'fancy_ranking':[], 'prestige_time_limit':[], 'enhanced_cattributes_eyes':[],
       'enhanced_cattributes_pattern':[], 'enhanced_cattributes_body':[],
       'enhanced_cattributes_mouth':[], 'enhanced_cattributes_colorprimary':[],
       'enhanced_cattributes_colorsecondary':[],
       'enhanced_cattributes_colortertiary':[], 'enhanced_cattributes_coloreyes':[],
       'status_cooldown_index':[], 'status_is_ready':[], 'status_is_gestating':[],
       'purrs_count':[], 'purrs_is_purred':[], 'watchlist_count':[],
       'watchlist_is_watchlisted':[]}


for d in dirs:
    file= codecs.open("html/"+ d, 'r',  encoding="utf8")
    try:
        parsed = json.loads(file.read())
        parsed = parsed['kitties']
    except:
        pass

    print(f"{d} DONE!", end = "\r")

    for i in range(100):
#             try: 
            # Price
            vals['USD'].append(int(parsed[i]['auction']['current_price'])/1000000000000000000)

            # Random
            vals['generation'].append(parsed[i]['generation'])        
            vals['color'].append(parsed[i]['color'])
            vals['is_fancy'].append(parsed[i]['is_fancy'])
            vals['is_exclusive'].append(parsed[i]['is_exclusive'])
            vals['fancy_type'].append(parsed[i]['fancy_type'])
            vals['is_prestige'].append(parsed[i]['is_prestige'])
            vals['prestige_type'].append(parsed[i]['prestige_type'])
            vals['prestige_ranking'].append(parsed[i]['prestige_ranking'])
            vals['fancy_ranking'].append(parsed[i]['fancy_ranking'])
            vals['prestige_time_limit'].append(parsed[i]['prestige_time_limit'])

            # Enhanced cattributes
            if parsed[i]['enhanced_cattributes'] != []:
                vals['enhanced_cattributes_eyes'].append(parsed[i]['enhanced_cattributes'][0]['description'])
                vals['enhanced_cattributes_pattern'].append(parsed[i]['enhanced_cattributes'][1]['description'])
                vals['enhanced_cattributes_body'].append(parsed[i]['enhanced_cattributes'][2]['description'])
                vals['enhanced_cattributes_mouth'].append(parsed[i]['enhanced_cattributes'][3]['description'])
                vals['enhanced_cattributes_colorprimary'].append(parsed[i]['enhanced_cattributes'][4]['description'])
                vals['enhanced_cattributes_colorsecondary'].append(parsed[i]['enhanced_cattributes'][5]['description'])
                vals['enhanced_cattributes_colortertiary'].append(parsed[i]['enhanced_cattributes'][6]['description'])
                vals['enhanced_cattributes_coloreyes'].append(parsed[i]['enhanced_cattributes'][7]['description'])
            else:
                vals['enhanced_cattributes_eyes'].append(None)
                vals['enhanced_cattributes_pattern'].append(None)
                vals['enhanced_cattributes_body'].append(None)
                vals['enhanced_cattributes_mouth'].append(None)
                vals['enhanced_cattributes_colorprimary'].append(None)
                vals['enhanced_cattributes_colorsecondary'].append(None)
                vals['enhanced_cattributes_colortertiary'].append(None)
                vals['enhanced_cattributes_coloreyes'].append(None)
                
            # Status
            vals['status_cooldown_index'].append(parsed[i]['status']['cooldown_index'])
            vals['status_is_ready'].append(parsed[i]['status']['is_ready'])
            vals['status_is_gestating'].append(parsed[i]['status']['is_gestating'])

            # Purrs
            vals['purrs_count'].append(parsed[i]['purrs']['count'])
            vals['purrs_is_purred'].append(parsed[i]['purrs']['is_purred'])

            # watchlist
            vals['watchlist_count'].append(parsed[i]['watchlist']['count'])
            vals['watchlist_is_watchlisted'].append(parsed[i]['watchlist']['is_watchlisted'])


#             print(f"we at: {k} out of {data.shape[0]}. Giving us percent complete of {round(100*k/int(data.shape[0]), 5)}%   ", end = "\r")

#             if k % 100 == 0:
#                 data.to_csv("Latest_kitties_{}.csv".format(date_format), index = False)
#             except:
#                 print("\n" + str(d))
        


In [80]:
for v in vals.keys():
    data[v] = vals[v]
#     print(len(vals[v]))

In [81]:
data.head()

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,,1637950,0.008,2,parakeet,False,False,None,False,None,NaN,NaN,None,happygokitty,chantilly,chronic,rorschach,cottoncandy,egyptiankohl,periwinkle,parakeet,1,True,False,1,False,0,False
1,,1591142,0.008,1,coralsunrise,False,False,None,False,None,NaN,NaN,None,totesbasic,soserious,savannah,wiley,bananacream,lemonade,icy,coralsunrise,0,True,False,0,False,0,False
2,,1340038,0.008,13,eclipse,True,False,TallyThePurrocious,False,None,NaN,928.0,None,None,None,None,None,None,None,None,None,6,True,False,0,False,0,False
3,,1592225,0.008,16,coralsunrise,True,False,Curdlin,False,None,NaN,1738.0,None,None,None,None,None,None,None,None,None,11,True,False,0,False,0,False
4,,1589771,0.008,12,cyan,True,False,Curdlin,False,None,NaN,1088.0,None,None,None,None,None,None,None,None,None,11,True,False,0,False,0,False


In [82]:
import datetime
date_format = datetime.datetime.now().isoformat().replace(':', "-")[:-7]

In [83]:
data.to_csv("Latest_kitties_{}.csv".format(date_format), index = False)

In [64]:
vals = {'generation': [],'color': [], 'is_fancy':[], 'is_exclusive':[],
       'fancy_type':[], 'is_prestige':[], 'prestige_type':[], 'prestige_ranking':[],
       'fancy_ranking':[], 'prestige_time_limit':[], 'enhanced_cattributes_eyes':[],
       'enhanced_cattributes_pattern':[], 'enhanced_cattributes_body':[],
       'enhanced_cattributes_mouth':[], 'enhanced_cattributes_colorprimary':[],
       'enhanced_cattributes_colorsecondary':[],
       'enhanced_cattributes_colortertiary':[], 'enhanced_cattributes_coloreyes':[],
       'status_cooldown_index':[], 'status_is_ready':[], 'status_is_gestating':[],
       'purrs_count':[], 'purrs_is_purred':[], 'watchlist_count':[],
       'watchlist_is_watchlisted':[]}

In [65]:
vals['generation']

[]

In [52]:
vals['test'].append(2)

In [53]:
vals['test']

[2, 2, 2]